# Проект по машинному обучению для текстов 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Проверка-целостности-данных-в-файле" data-toc-modified-id="Проверка-целостности-данных-в-файле-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Проверка целостности данных в файле</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Логистическая-ригресия" data-toc-modified-id="Логистическая-ригресия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая ригресия</a></span></li><li><span><a href="#Случайный-лес-решающих-деревьев" data-toc-modified-id="Случайный-лес-решающих-деревьев-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес решающих деревьев</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#Анализ-полученных-результатов" data-toc-modified-id="Анализ-полученных-результатов-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Анализ полученных результатов</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-тестовых-данных" data-toc-modified-id="Проверка-лучшей-модели-на-тестовых-данных-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Проверка лучшей модели на тестовых данных</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

**Описание проекта**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделайть выводы.

Для выполнения проекта была выбрана библиотека BERT/

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. В столбеце *text* содержится текст комментария, а *toxic* — целевой признак.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import tqdm
from tqdm import notebook
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

In [2]:
# загрузка данных
!wget https://code.s3.yandex.net/datasets/toxic_comments.csv

--2023-03-31 09:21:55--  https://code.s3.yandex.net/datasets/toxic_comments.csv
Resolving code.s3.yandex.net (code.s3.yandex.net)... 93.158.134.158, 2a02:6b8::2:158
Connecting to code.s3.yandex.net (code.s3.yandex.net)|93.158.134.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64950309 (62M) [text/csv]
Saving to: ‘toxic_comments.csv.12’

     0K .......... .......... .......... .......... ..........  0%  160M 0s
    50K .......... .......... .......... .......... ..........  0% 98.5M 1s
   100K .......... .......... .......... .......... ..........  0% 90.0M 1s
   150K .......... .......... .......... .......... ..........  0%  274M 0s
   200K .......... .......... .......... .......... ..........  0%  193M 0s
   250K .......... .......... .......... .......... ..........  0%  125M 0s
   300K .......... .......... .......... .......... ..........  0% 79.6M 1s
   350K .......... .......... .......... .......... ..........  0% 91.3M 1s
   400K .......... ...

### Проверка целостности данных в файле

In [3]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])
except:
    data = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0])

In [4]:
data.sample(15)

,text,toxic
49369,I propose that you mind you own business and k...,0
156625,Contested deletion \n\nThis page is not unambi...,0
25230,", 9 April 2007 (UTC)\n the reliability of JN s...",0
140282,Perhaps the overwhelming evidence points to Ko...,0
103729,"The t and the h are more or less silent, givin...",0
100377,"""\nHey, thanks for adding a link to the Dink l...",0
152253,"""\n\n Quit removing the """"status unclear"""" tab...",0
154092,"article. Melodic death metal may have evolved,...",0
11635,Warning Me\n\nIf you dare to warn me I dont re...,1
51739,Windows RT Edit War (sigh) \n\nPlease contribu...,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
text     159292 non-null object
toxic    159292 non-null int64
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
data['toxic'].mean()

0.10161213369158527

В предоставленном файле содержатся текстовые комментарии на английском языке(text) и тональность отзыва(toxic). В файле содержится 159,5 тыс строк для обучения и проверки модели. Всего 10,2% от всех коментариев являются токсичными. Это может повлиять на результаты работы модели.

In [7]:
data_toxic_1 = data[data['toxic']==1]

In [8]:
data_toxic_0 = data[data['toxic']==0]

### Подготовка данных

Для подготовки данных воспользуемся моделью BERT, обученной на английском языке. Для дальнейшей обработки была выбрана модель, которая была предобучена для определения тональности текста(toxic-bert).

In [9]:
%time
tokenizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs


In [10]:
%%time
tokenized = data['text'].apply(lambda x: 
                               tokenizer.encode(x,
                                        add_special_tokens=True)) 

Token indices sequence length is longer than the specified maximum sequence length for this model (631 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 49.8 s, sys: 1.2 s, total: 51 s
Wall time: 53.1 s


Найдем длину самого длинного отзыва и исключим слишком длинные(аномальные) отзывы из данных.

In [11]:
token_max = tokenized.apply(len).max()
print('Макмисальная длина строки -', token_max)

if token_max > 512:
    ind = tokenized[tokenized.apply(len) > 512].index
    print('Количество строк длиннее 512 -', \
                      tokenized[tokenized.apply(len) > 512].count())
    print('Из них токсичных -', \
                     '%.3f' %(data['toxic'][ind].mean() * 10), '%.')
    print('Исключим аномально длинные строки.')
    
    tokenized = tokenized.drop(index= ind)
    data_clean = data.drop(index= ind)
    n = 512
else:
    n = token_max

Макмисальная длина строки - 4950
Количество строк длиннее 512 - 3503
Из них токсичных - 0.979 %.
Исключим аномально длинные строки.


In [12]:
padded = np.array([i + [0]*(n - len(i)) for i in tokenized.values])

In [13]:
attention_mask = np.where(padded != 0, 1, 0)

In [14]:
config = transformers.BertConfig.from_json_file(
           'conversational_cased_L-12_H-768_A-12_pt_v1/config.json')
model = transformers.AutoModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Failed to deserialize variable 'model'. Run the following code to delete it:
  del_datasphere_variables('model')
Traceback (most recent call last):
  File "/kernel/lib/python3.8/site-packages/ml_kernel/state/state_protocol.py", line 283, in _load_component
    value = unpickler.load()
  File "/usr/local/lib/python3.8/dist-packages/torch/storage.py", line 161, in _load_from_bytes
    return torch.load(

In [15]:
#!g2.mig
if torch.cuda.is_available():
    model.to('cuda')

In [16]:
#!g2.mig
%%time
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to('cuda') 
    attention_mask_batch = torch.LongTensor(
                                attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')
        
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())


CPU times: user 29min 49s, sys: 21min 42s, total: 51min 32s
Wall time: 51min 33s


Предобработка данных завершена. Теперь можем перейти к обучению моделей и выбору лучшей.

## Обучение моделей

Для обучения выберим 3 модели - простую логистическую регрессию, решающий лес и модель с бустингом. Для обучения воспользуемся кросс-валидацией, а за метрику качества примем F1. Данная метрика является оптимальным показателем между полнотой и точностью предсказаний. Так как в полученных данных был обнаружен дисбаланс классов, при обученнии модели будем учитывать вес класов.

Перед обучением моделей разделим выделенные данные на обучающие и тестовые выборки.

In [17]:
features = np.concatenate(embeddings)
target = data_clean['toxic']

In [18]:
features_train, features_test, \
target_train, target_test = train_test_split(features, target, 
                                             test_size=0.10, 
                                             stratify=target, 
                                             random_state=12345) 

In [19]:
# функция обучения модели
def model_scorer(model_for_gcv, param):
       
    grid_CV = GridSearchCV(estimator = model_for_gcv,
                           param_grid = param, 
                           n_jobs=-1, 
                           scoring='f1', 
                           cv=5)

    grid_CV.fit(features_train, target_train)

    model_gcv = grid_CV.best_estimator_
    f_1 = grid_CV.best_score_
    
    print('Модель проверена. Лучшее значение'
           + ' по результатам кросс-валидации -',
          '%.3f' %f_1, '.')
    
    return model_gcv, f_1

### Логистическая ригресия
Начнем с самой простой модели - логистической регрессии.

In [20]:
#!c1.32
%%time
param_lr = {}
model_lr, f1_lr = model_scorer(LogisticRegression(random_state=12345,
                                                  class_weight='balanced',
                                                  solver='lbfgs', 
                                                  max_iter=10000), 
                               param_lr)

Модель проверена. Лучшее значение по результатам кросс-валидации - 0.928 .
CPU times: user 18min 25s, sys: 38min 40s, total: 57min 6s
Wall time: 4min 37s


### Случайный лес решающих деревьев
Начнем с самой простой модели - логистической регрессии.

In [21]:
#!c1.32
%%time
param_rfc = {'n_estimators' : range(30, 131, 20),
             'max_depth' : range(5, 21, 5) 
            }
model_rfc, f1_rfc = model_scorer(
                         RandomForestClassifier(class_weight='balanced',
                                                random_state=12345),
                                 param_rfc)

Модель проверена. Лучшее значение по результатам кросс-валидации - 0.945 .
CPU times: user 13min 37s, sys: 2min 20s, total: 15min 57s
Wall time: 44min 34s


### LGBMClassifier
Последним обучим классификатор из библиотеки LGBM.

In [22]:
#!c1.32
%%time
param_lgbm = {'max_depth': range(5, 16, 5),
              'learning_rate': [0.1, 0.3]
             }
model_lgbm, f1_lgbm = model_scorer(
                         LGBMClassifier(boosting_type='gbdt', 
                                        class_weight='balanced',
                                        num_leaves= 25,
                                        random_state=12345), 
                         param_lgbm)

Модель проверена. Лучшее значение по результатам кросс-валидации - 0.936 .
CPU times: user 3min 40s, sys: 31.2 s, total: 4min 12s
Wall time: 2min 22s


### Анализ полученных результатов

Сведем полученные результары метрики по кросс-валидации в единую таблицу.

In [23]:
pd.DataFrame([
    ['Логистическая регрессия', round(f1_lr, 3)],
    ['Случайный лес решающих деревьев', round(f1_rfc, 3)],
    ['LightGBM', round(f1_lgbm, 3)],
              ], columns=['Модель', 'F1'])

,Модель,F1
0,Логистическая регрессия,0.928
1,Случайный лес решающих деревьев,0.945
2,LightGBM,0.936


Лучший результат оказался у модели случайного леса. Проверим лучшею модель на тестовых данных.

### Проверка лучшей модели на тестовых данных

In [24]:
def tester(model_for_test):
    pred_test = model_for_test.predict(features_test)
    f_1 = f1_score(target_test, pred_test)
    print('F1 тестовых данных =', '%.3f' %f_1)

In [25]:
tester(model_rfc)

F1 тестовых данных = 0.941


Метрика качества на тестовых данных вполне удовлетворяет условию F1 > 0.75, а значит модель готова к использованию.

## Выводы

На основании предоставленных данных была разработана модель для определения токсичности комментариев.

Основные этапы работы:
- рассмотрение полученных данных
- предобработка полученных текстов с помощю библиотеки BERT( предобученный для токсичных текстов)
- разделение данных на обучающие и тестовые
- обучение моделей методом кросс-валлидации
- проверка лучшей модели на тестовых данных

На этапе рассмотрения полученных данных был обнаружен сильный дисбаланс между токсичными и нетоксичными комментариями. Так как токсичных комментариев всего 10%, было принято решение о уменьшении количества нетоксичных комментариев в данных до 20% от общего количества.

С помощью библиотеки BERT была произведена обработка текстов, в результате которой были получены эмбендинги для последующего обучения моделей.

Для обучения были выбаны 4 модели, на которых были получены следующие данные.

|Модель|F1|
|:----|:----:|
|Логистическая регрессия | 0.928 |
|Случайный лес решающих деревьев | 0.945 |
|LightGBM | 0.936 |

Как видно из таблицы, лучшие результаты получены у модели - случайный лес решающих деревьев. 

На тестовой выборке у лучшей модели было получено следующее значение метрики качества:

- обучающие данные - 0.945
- тестовые данные - 0.941

Как видно, модель успешно предсказывает значения и готова к дальнейшему использованию.